# Find Optimal Photometry for HD163466 in Y3 filter
=============================================================================

- creation 08/07/2016
- author Sylvie Dagoret-Campagne


In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

from astropy.table import Table
from astropy.table import Column

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [51]:
import photutils
from astropy.stats import sigma_clipped_stats
from photutils import daofind
from photutils import CircularAperture,CircularAnnulus
from photutils import aperture_photometry
from astropy.table import hstack
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

In [7]:
from scipy import stats 
from scipy import ndimage
import os
from datetime import datetime, timedelta

In [8]:
from photutils.background import Background2D

In [9]:
import libMonocamBaseImages           # my tool library written to do that CCD reduction

In [10]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-07-08 21:27:54.965791
fits date format for the analysis :  2016-07-08T21:27:54


In [11]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    weights=np.where(weights>0,weights,0)
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (average, np.sqrt(variance))

## Definitions and Constants

In [12]:
object_name='HD163466_y3'

### input files

In [13]:
path='./HD163466_y3'
rootfilename='AssScImHD163466_y3_' 


NumStart=1
NumStop=10

### output file (table)

In [38]:
outputtablefile='HD163466_y3_1-10_TablePhotometry.fits'

### make the filelist

In [14]:
filelist=libMonocamBaseImages.BuildFilelist(path,rootfilename,start=NumStart,stop=NumStop,nbchar=1)

In [17]:
filelist

['./HD163466_y3/AssScImHD163466_y3_1.fits',
 './HD163466_y3/AssScImHD163466_y3_2.fits',
 './HD163466_y3/AssScImHD163466_y3_3.fits',
 './HD163466_y3/AssScImHD163466_y3_4.fits',
 './HD163466_y3/AssScImHD163466_y3_5.fits',
 './HD163466_y3/AssScImHD163466_y3_6.fits',
 './HD163466_y3/AssScImHD163466_y3_7.fits',
 './HD163466_y3/AssScImHD163466_y3_8.fits',
 './HD163466_y3/AssScImHD163466_y3_9.fits',
 './HD163466_y3/AssScImHD163466_y3_10.fits']

## Read Input files

In [15]:
allchannelallsciimages = []  # list of 16 lists of images series 
exposures_list = []        # sequential list of the exposures of the sky flats 
header_list = []           # list of headers
data_list = []
time_list = []                # date and time
basefile_list = []         # basefilename
dateobs_list = [] 
# get the primary block headers:
for image_file in filelist: 
    print image_file
    hdu_list = fits.open(image_file)
    basefile_list.append(os.path.basename(image_file))
    header=hdu_list[0].header
    exposure=header['EXPOSURE']
    exposures_list.append(exposure)
    dateobs_list.append(header['DATE-OBS'])
    header_list.append(header)
    data=ccdproc.CCDData.read(image_file, hdu=0,unit='adu') 
    data_list.append(data)

./HD163466_y3/AssScImHD163466_y3_1.fits
./HD163466_y3/AssScImHD163466_y3_2.fits
./HD163466_y3/AssScImHD163466_y3_3.fits
./HD163466_y3/AssScImHD163466_y3_4.fits
./HD163466_y3/AssScImHD163466_y3_5.fits
./HD163466_y3/AssScImHD163466_y3_6.fits
./HD163466_y3/AssScImHD163466_y3_7.fits
./HD163466_y3/AssScImHD163466_y3_8.fits
./HD163466_y3/AssScImHD163466_y3_9.fits
./HD163466_y3/AssScImHD163466_y3_10.fits


In [19]:
#basefile_list

## For control

uncomment for control

In [20]:
index=0

In [21]:
#print exposures_list[index]

In [22]:
#header_list[index]

In [23]:
#plt.imshow(data_list[index])

In [24]:
#bkg= Background2D(data_list[index], (100, 100), filter_size=(3, 3),method='median')

In [25]:
#data_list[index].data-bkg.background

## Background subtraction

In [16]:
correctedimage_list = []

In [17]:
for idx,data in enumerate(data_list):
    bkg= Background2D(data, (100, 100), filter_size=(3, 3),method='median')
    newimage=(data-bkg.background)/exposure_list(idx)
    correctedimage_list.append(newimage)

## Crop the images

In [18]:
NBIMAGES=len(correctedimage_list)

In [19]:
wpix=200

In [20]:
star2D_list = []
position_list = []
sigmapix_list = []

In [43]:
# loop on images
for idx,image in enumerate(correctedimage_list):
    (y0,x0)=np.where(image == image.max())
    print 'star',idx,' at (x0,y0) =',x0,',',y0
    the_star2D=image[y0-wpix:y0+wpix,x0-wpix:x0+wpix]
    star2D_list.append(the_star2D)
    thepsfX=the_star2D.sum(axis=0)
    thepsfY=the_star2D.sum(axis=1)
    (mx,sigx)=weighted_avg_and_std(np.arange(thepsfX.shape[0]),thepsfX)
    (my,sigy)=weighted_avg_and_std(np.arange(thepsfY.shape[0]),thepsfY)
    position_list.append((my,mx))
    sigmapix_list.append((sigy,sigx))
    

star 0  at (x0,y0) = [2110] , [1886]
star 1  at (x0,y0) = [2107] , [1881]
star 2  at (x0,y0) = [2105] , [1881]


/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/ipykernel/__main__.py:5: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


star 3  at (x0,y0) = [2107] , [1883]
star 4  at (x0,y0) = [2113] , [1874]
star 5  at (x0,y0) = [2108] , [1881]
star 6  at (x0,y0) = [2106] , [1882]
star 7  at (x0,y0) = [2105] , [1881]
star 8  at (x0,y0) = [2105] , [1881]
star 9  at (x0,y0) = [2104] , [1881]


## Compute photometry

In [46]:
phot_table_list = []

In [53]:
for idx,star_im in enumerate(star2D_list):

    print ' ========================================================='    
    print 'photometry for image ',idx
    print ' ========================================================='  
    
    the_sigmapix=max(sigmapix_list[idx])
    the_position=position_list[idx]
    
    star_aperture=CircularAperture(the_position, 3*the_sigmapix)
    rawflux_table = aperture_photometry(star_im, star_aperture)
    
    annulus_apertures = CircularAnnulus(the_position, r_in=5*the_sigmapix, r_out=7*the_sigmapix)
    bkgflux_table = aperture_photometry(star_im, annulus_apertures)
    
    phot_table = hstack([rawflux_table, bkgflux_table], table_names=['raw', 'bkg'])
    
    bkg_mean = phot_table['aperture_sum_bkg'] / annulus_apertures.area()
    
    bkg_sum = bkg_mean * star_aperture.area()
    final_sum = phot_table['aperture_sum_raw'] - bkg_sum
    phot_table['residual_aperture_sum'] = final_sum
    print(phot_table['residual_aperture_sum']) 
    
    phot_table_list.append(phot_table)
    
    
    

photometry for image  0
residual_aperture_sum
                     
---------------------
        152480.335769
photometry for image  1
residual_aperture_sum
                     
---------------------
        768883.946211
photometry for image  2
residual_aperture_sum
                     
---------------------
         771923.14588
photometry for image  3
residual_aperture_sum
                     
---------------------
        771643.100095
photometry for image  4
residual_aperture_sum
                     
---------------------
        462751.870602
photometry for image  5
residual_aperture_sum
                     
---------------------
        462800.677314
photometry for image  6
residual_aperture_sum
                     
---------------------
        465137.324654
photometry for image  7
residual_aperture_sum
                     
---------------------
        463611.861144
photometry for image  8
residual_aperture_sum
                     
---------------------
        465038

In [54]:
phot_table_list[0]

aperture_sum_raw,xcenter_raw [1],ycenter_raw [1],aperture_sum_bkg,xcenter_bkg [1],ycenter_bkg [1],residual_aperture_sum
,,,,,,
float64,float64,float64,float64,float64,float64,float64
152496.623345,196.349926657,196.462089774,43.4335359462,196.349926657,196.462089774,152480.335769


## Create astropy table

In [33]:
t=Table(rows=image_psf,names=('num','nbstars','prfx','pfry','sig_prfx','sig_prfy'),dtype=('i4','i4','f8','f8','f8','f8'))

In [34]:
t

num,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,int32,float64,float64,float64,float64
0,7,3.14462939981,3.41466663301,0.0624903435569,0.0399877500438
1,23,3.23590764548,3.67168661807,0.210812621801,0.313203469327
2,30,3.28439425917,3.44808452428,0.153086049741,0.109240066938
3,29,3.07852576115,3.39865028239,0.0743265241544,0.182716405574
4,9,5.01067679435,5.12760526598,0.431963018433,0.931127413941
5,4,3.66881377389,3.71438578859,0.125757386795,0.0576096104765
6,15,2.62049631174,2.75934903687,0.053654463766,0.0477390482732
7,12,3.02620244552,2.97225160347,0.0890701171525,0.163930894688
8,15,2.85053312944,2.90077455,0.076245483463,0.038651577062


In [35]:
expo = Column(exposures_list, name='exposure')
file = Column(basefile_list, name='file')
time = Column(dateobs_list,name='time')

In [36]:
t.add_column(expo, index=1)
t.add_column(time, index=1)
t.add_column(file, index=1)

In [37]:
t

num,file,time,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,str26,str23,float64,int32,float64,float64,float64,float64
0,AssScImHD163466_y3_1.fits,2016-05-10T08:31:46.221,1.0,7,3.14462939981,3.41466663301,0.0624903435569,0.0399877500438
1,AssScImHD163466_y3_2.fits,2016-05-10T08:33:17.773,5.0,23,3.23590764548,3.67168661807,0.210812621801,0.313203469327
2,AssScImHD163466_y3_3.fits,2016-05-10T08:33:27.174,5.0,30,3.28439425917,3.44808452428,0.153086049741,0.109240066938
3,AssScImHD163466_y3_4.fits,2016-05-10T08:33:36.166,5.0,29,3.07852576115,3.39865028239,0.0743265241544,0.182716405574
4,AssScImHD163466_y3_5.fits,2016-05-10T08:38:53.776,3.0,9,5.01067679435,5.12760526598,0.431963018433,0.931127413941
5,AssScImHD163466_y3_6.fits,2016-05-10T08:39:35.943,3.0,4,3.66881377389,3.71438578859,0.125757386795,0.0576096104765
6,AssScImHD163466_y3_7.fits,2016-05-10T08:40:17.637,3.0,15,2.62049631174,2.75934903687,0.053654463766,0.0477390482732
7,AssScImHD163466_y3_8.fits,2016-05-10T08:41:40.854,3.0,12,3.02620244552,2.97225160347,0.0890701171525,0.163930894688
8,AssScImHD163466_y3_9.fits,2016-05-10T08:41:47.875,3.0,15,2.85053312944,2.90077455,0.076245483463,0.038651577062


## Write output file

In [38]:
t.write(outputtablefile,format='fits')

## Pandas

In [39]:
df=t.to_pandas()

In [40]:
df.describe()

,num,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,4.50000,3.400000,16.400000,3.266613,3.427067,0.137860,0.195532
std,3.02765,1.264911,8.946756,0.681054,0.689866,0.113786,0.272582
min,0.00000,1.000000,4.000000,2.620496,2.759349,0.053654,0.038652
25%,2.25000,3.000000,9.750000,2.894450,2.918644,0.074806,0.050207
50%,4.50000,3.000000,15.000000,3.111578,3.406658,0.095132,0.090178
75%,6.75000,4.500000,22.250000,3.272273,3.615786,0.146254,0.178020
max,9.00000,5.000000,30.000000,5.010677,5.127605,0.431963,0.931127
